<a href="https://colab.research.google.com/github/Jyunci/AIoT_cute/blob/main/tool_wear_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.1  刀具磨耗預測
使用  Temporal Convolutional Network  (TCN) 模型，預測端銑刀是否磨損 (Worn)。

資料來源：公開資料集 Kaggle 平台 --- CNC Mill Tool Wear。

連結：https://www.kaggle.com/shasun/tool-wear-detection-in-cnc-mill

## 前置作業 - 函式庫環境設置

In [1]:
# 共通函式庫
import numpy as np
import os
import pandas as pd
import scipy.stats as stats
from sklearn.preprocessing import LabelEncoder

In [2]:
!pip install keras-tcn

     |████████████████████████████████| 378kB 13.1MB/s 
     |████████████████████████████████| 51kB 8.1MB/s 
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3


In [3]:
from tensorflow.keras import Model, Input
from tensorflow.keras.datasets import imdb
from tensorflow.keras.layers import Dense, Dropout, Embedding
from tensorflow.keras.preprocessing import sequence
from tcn import TCN

## 前置作業 - 資料下載  (從 github clone 檔案)

In [4]:
!git clone https://github.com/Jyunci/AIoT_cute.git

Cloning into 'AIoT_cute'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 101 (delta 9), reused 0 (delta 0), pack-reused 81
Receiving objects: 100% (101/101), 4.46 MiB | 23.93 MiB/s, done.
Resolving deltas: 100% (40/40), done.


In [5]:
#  讀取 'train_set.csv'  檢視資料
df_train = pd.read_csv('./AIoT_cute/cnc_tool/train_set.csv')  
df_train.head(5)

,Unnamed: 0,tool_condition,feedrate,clamp_pressure,X1_ActualPosition,X1_ActualVelocity,X1_ActualAcceleration,X1_CommandPosition,X1_CommandVelocity,X1_CommandAcceleration,X1_CurrentFeedback,X1_DCBusVoltage,X1_OutputCurrent,X1_OutputVoltage,X1_OutputPower,Y1_ActualPosition,Y1_ActualVelocity,Y1_ActualAcceleration,Y1_CommandPosition,Y1_CommandVelocity,Y1_CommandAcceleration,Y1_CurrentFeedback,Y1_DCBusVoltage,Y1_OutputCurrent,Y1_OutputVoltage,Y1_OutputPower,Z1_ActualPosition,Z1_ActualVelocity,Z1_ActualAcceleration,Z1_CommandPosition,Z1_CommandVelocity,Z1_CommandAcceleration,Z1_CurrentFeedback,Z1_DCBusVoltage,Z1_OutputCurrent,Z1_OutputVoltage,S1_ActualPosition,S1_ActualVelocity,S1_ActualAcceleration,S1_CommandPosition,S1_CommandVelocity,S1_CommandAcceleration,S1_CurrentFeedback,S1_DCBusVoltage,S1_OutputCurrent,S1_OutputVoltage,S1_OutputPower,S1_SystemInertia,M1_CURRENT_PROGRAM_NUMBER,M1_sequence_number,M1_CURRENT_FEEDRATE,Machining_Process
0,0,unworn,6,4.0,198.0,0.0,0.00,198.0,0.0,0.000000,0.18,0.0207,329.0,2.77,-0.000001,158.0,-0.025,-6.25,158.0,0.0,0.000000,0.539,0.0167,328.0,1.84,6.430000e-07,119.0,0.0,0.00,119.0,0.0,0.000000,0.0,0.0,0.0,0.0,-361.0,0.001,0.250,-361.0,0.0,0.0,0.524,2.740000e-19,329.0,0.0,6.960000e-07,12.0,1.0,0.0,50.0,Starting
1,1,unworn,6,4.0,198.0,-10.8,-350.00,198.0,-13.6,-358.000000,-10.90,0.1860,328.0,23.30,0.004480,158.0,-19.800,-750.00,157.0,-24.6,-647.000000,-14.500,0.2810,325.0,37.80,1.260000e-02,119.0,-20.3,-712.00,118.0,-25.6,-674.000000,0.0,0.0,0.0,0.0,-361.0,0.000,0.250,-361.0,0.0,0.0,-0.288,2.740000e-19,328.0,0.0,-5.270000e-07,12.0,1.0,4.0,50.0,Prep
2,2,unworn,6,4.0,196.0,-17.8,-6.25,196.0,-17.9,-0.000095,-8.59,0.1400,328.0,30.60,0.005330,154.0,-32.500,0.00,154.0,-32.3,-0.000095,-7.790,0.1390,327.0,49.40,9.430000e-03,115.0,-33.7,37.50,115.0,-33.7,-0.000095,0.0,0.0,0.0,0.0,-361.0,0.000,-0.438,-361.0,0.0,0.0,0.524,2.740000e-19,328.0,0.0,9.100000e-07,12.0,1.0,7.0,50.0,Prep
3,3,unworn,6,4.0,194.0,-18.0,0.00,194.0,-17.9,-0.000095,-6.11,0.1300,327.0,30.30,0.004890,151.0,-32.600,-62.50,151.0,-32.3,-0.000095,-8.130,0.1560,325.0,47.60,1.050000e-02,112.0,-33.7,-6.25,112.0,-33.7,0.000000,0.0,0.0,0.0,0.0,-361.0,-0.001,-0.250,-361.0,0.0,0.0,-0.180,2.740000e-19,328.0,0.0,1.070000e-06,12.0,1.0,7.0,50.0,Prep
4,4,unworn,6,4.0,193.0,-17.9,-18.80,192.0,-17.9,0.000095,-5.70,0.1140,328.0,30.50,0.004250,148.0,-32.000,138.00,148.0,-32.3,0.000095,-13.800,0.2020,326.0,47.10,1.350000e-02,109.0,-33.6,18.80,108.0,-33.7,0.000000,0.0,0.0,0.0,0.0,-361.0,0.000,-0.188,-361.0,0.0,0.0,0.226,2.740000e-19,328.0,0.0,1.960000e-06,12.0,1.0,7.0,50.0,Prep


In [6]:
print('訓練集大小: ', df_train.shape)
tool_df = df_train['tool_condition'].value_counts(normalize=True)
print(tool_df)

訓練集大小:  (17793, 52)
worn      0.548474
unworn    0.451526
Name: tool_condition, dtype: float64


In [7]:
#  讀取 'test_set.csv'  檢視資料
df_test = pd.read_csv('./AIoT_cute/cnc_tool/test_set.csv')
df_test.head(5)

,Unnamed: 0,tool_condition,feedrate,clamp_pressure,X1_ActualPosition,X1_ActualVelocity,X1_ActualAcceleration,X1_CommandPosition,X1_CommandVelocity,X1_CommandAcceleration,X1_CurrentFeedback,X1_DCBusVoltage,X1_OutputCurrent,X1_OutputVoltage,X1_OutputPower,Y1_ActualPosition,Y1_ActualVelocity,Y1_ActualAcceleration,Y1_CommandPosition,Y1_CommandVelocity,Y1_CommandAcceleration,Y1_CurrentFeedback,Y1_DCBusVoltage,Y1_OutputCurrent,Y1_OutputVoltage,Y1_OutputPower,Z1_ActualPosition,Z1_ActualVelocity,Z1_ActualAcceleration,Z1_CommandPosition,Z1_CommandVelocity,Z1_CommandAcceleration,Z1_CurrentFeedback,Z1_DCBusVoltage,Z1_OutputCurrent,Z1_OutputVoltage,S1_ActualPosition,S1_ActualVelocity,S1_ActualAcceleration,S1_CommandPosition,S1_CommandVelocity,S1_CommandAcceleration,S1_CurrentFeedback,S1_DCBusVoltage,S1_OutputCurrent,S1_OutputVoltage,S1_OutputPower,S1_SystemInertia,M1_CURRENT_PROGRAM_NUMBER,M1_sequence_number,M1_CURRENT_FEEDRATE,Machining_Process
0,0,unworn,20,4.0,198.0,0.0,0.0,198.0,0.0,0.0,-0.284,2.790000e-19,329.0,0.0,0.000000e+00,158.0,0.0,0.0,158.0,0.0,0.0,-0.0839,2.680000e-19,333.0,0.0,-1.470000e-06,119.0,0.000,0.00,119.0,0.0,0.0,0.0,0.0,0.0,0.0,-1160.0,0.00100,0.687,-1160.0,0.0,0.0,-1.860,0.000000e+00,332.0,0.0,0.000000e+00,12.0,1.0,2.0,50.0,Prep
1,1,unworn,20,4.0,198.0,0.0,0.0,198.0,0.0,0.0,-0.284,2.790000e-19,329.0,0.0,0.000000e+00,158.0,0.0,0.0,158.0,0.0,0.0,-0.0839,2.680000e-19,333.0,0.0,-1.470000e-06,119.0,0.000,0.00,119.0,0.0,0.0,0.0,0.0,0.0,0.0,-1160.0,0.00100,0.687,-1160.0,0.0,0.0,-1.860,0.000000e+00,332.0,0.0,0.000000e+00,12.0,0.0,0.0,50.0,Prep
2,2,unworn,20,4.0,198.0,0.0,0.0,198.0,0.0,0.0,1.170,2.780000e-19,325.0,0.0,9.160000e-07,158.0,0.0,0.0,158.0,0.0,0.0,-0.7090,2.690000e-19,327.0,0.0,6.770000e-07,119.0,0.000,0.00,119.0,0.0,0.0,0.0,0.0,0.0,0.0,905.0,0.00025,0.875,905.0,0.0,0.0,-0.176,2.780000e-19,324.0,0.0,-8.590000e-07,12.0,0.0,0.0,50.0,Prep
3,3,unworn,20,4.0,198.0,0.0,0.0,198.0,0.0,0.0,-0.230,2.790000e-19,329.0,0.0,-2.300000e-07,158.0,0.0,0.0,158.0,0.0,0.0,-1.3400,2.680000e-19,333.0,0.0,0.000000e+00,119.0,-0.025,-6.25,119.0,0.0,0.0,0.0,0.0,0.0,0.0,-1160.0,0.00075,0.188,-1160.0,0.0,0.0,-1.170,0.000000e+00,332.0,0.0,-6.290000e-06,12.0,4.0,0.0,50.0,Prep
4,4,unworn,20,4.0,198.0,0.0,0.0,198.0,0.0,0.0,-0.230,2.790000e-19,329.0,0.0,-2.300000e-07,158.0,0.0,0.0,158.0,0.0,0.0,-1.3400,2.680000e-19,333.0,0.0,0.000000e+00,119.0,-0.025,-6.25,119.0,0.0,0.0,0.0,0.0,0.0,0.0,-1160.0,0.00075,0.188,-1160.0,0.0,0.0,-1.170,0.000000e+00,332.0,0.0,-6.290000e-06,12.0,0.0,0.0,50.0,Prep


In [8]:
print('測試集大小: ', df_test.shape)
tool_df = df_test['tool_condition'].value_counts(normalize=True)
print(tool_df)

測試集大小:  (7493, 52)
unworn    0.526358
worn      0.473642
Name: tool_condition, dtype: float64


### 準備  train_set/test_set

In [92]:
# Train_set
# 忽略第一二欄與最後一欄
X_train = df_train.iloc[:, 2:-1]
# 學習目標為第二攔
y_train = df_train.iloc[:, 1]

In [93]:
print(X_train.shape)
print(y_train.shape)

(17793, 49)
(17793,)


In [94]:
X_train.head(5)

,feedrate,clamp_pressure,X1_ActualPosition,X1_ActualVelocity,X1_ActualAcceleration,X1_CommandPosition,X1_CommandVelocity,X1_CommandAcceleration,X1_CurrentFeedback,X1_DCBusVoltage,X1_OutputCurrent,X1_OutputVoltage,X1_OutputPower,Y1_ActualPosition,Y1_ActualVelocity,Y1_ActualAcceleration,Y1_CommandPosition,Y1_CommandVelocity,Y1_CommandAcceleration,Y1_CurrentFeedback,Y1_DCBusVoltage,Y1_OutputCurrent,Y1_OutputVoltage,Y1_OutputPower,Z1_ActualPosition,Z1_ActualVelocity,Z1_ActualAcceleration,Z1_CommandPosition,Z1_CommandVelocity,Z1_CommandAcceleration,Z1_CurrentFeedback,Z1_DCBusVoltage,Z1_OutputCurrent,Z1_OutputVoltage,S1_ActualPosition,S1_ActualVelocity,S1_ActualAcceleration,S1_CommandPosition,S1_CommandVelocity,S1_CommandAcceleration,S1_CurrentFeedback,S1_DCBusVoltage,S1_OutputCurrent,S1_OutputVoltage,S1_OutputPower,S1_SystemInertia,M1_CURRENT_PROGRAM_NUMBER,M1_sequence_number,M1_CURRENT_FEEDRATE
0,6,4.0,198.0,0.0,0.00,198.0,0.0,0.000000,0.18,0.0207,329.0,2.77,-0.000001,158.0,-0.025,-6.25,158.0,0.0,0.000000,0.539,0.0167,328.0,1.84,6.430000e-07,119.0,0.0,0.00,119.0,0.0,0.000000,0.0,0.0,0.0,0.0,-361.0,0.001,0.250,-361.0,0.0,0.0,0.524,2.740000e-19,329.0,0.0,6.960000e-07,12.0,1.0,0.0,50.0
1,6,4.0,198.0,-10.8,-350.00,198.0,-13.6,-358.000000,-10.90,0.1860,328.0,23.30,0.004480,158.0,-19.800,-750.00,157.0,-24.6,-647.000000,-14.500,0.2810,325.0,37.80,1.260000e-02,119.0,-20.3,-712.00,118.0,-25.6,-674.000000,0.0,0.0,0.0,0.0,-361.0,0.000,0.250,-361.0,0.0,0.0,-0.288,2.740000e-19,328.0,0.0,-5.270000e-07,12.0,1.0,4.0,50.0
2,6,4.0,196.0,-17.8,-6.25,196.0,-17.9,-0.000095,-8.59,0.1400,328.0,30.60,0.005330,154.0,-32.500,0.00,154.0,-32.3,-0.000095,-7.790,0.1390,327.0,49.40,9.430000e-03,115.0,-33.7,37.50,115.0,-33.7,-0.000095,0.0,0.0,0.0,0.0,-361.0,0.000,-0.438,-361.0,0.0,0.0,0.524,2.740000e-19,328.0,0.0,9.100000e-07,12.0,1.0,7.0,50.0
3,6,4.0,194.0,-18.0,0.00,194.0,-17.9,-0.000095,-6.11,0.1300,327.0,30.30,0.004890,151.0,-32.600,-62.50,151.0,-32.3,-0.000095,-8.130,0.1560,325.0,47.60,1.050000e-02,112.0,-33.7,-6.25,112.0,-33.7,0.000000,0.0,0.0,0.0,0.0,-361.0,-0.001,-0.250,-361.0,0.0,0.0,-0.180,2.740000e-19,328.0,0.0,1.070000e-06,12.0,1.0,7.0,50.0
4,6,4.0,193.0,-17.9,-18.80,192.0,-17.9,0.000095,-5.70,0.1140,328.0,30.50,0.004250,148.0,-32.000,138.00,148.0,-32.3,0.000095,-13.800,0.2020,326.0,47.10,1.350000e-02,109.0,-33.6,18.80,108.0,-33.7,0.000000,0.0,0.0,0.0,0.0,-361.0,0.000,-0.188,-361.0,0.0,0.0,0.226,2.740000e-19,328.0,0.0,1.960000e-06,12.0,1.0,7.0,50.0


In [95]:
# Test_set
# 忽略第一二欄與最後一欄
# 學習目標為第二攔
X_test = df_test.iloc[:, 2:-1]
y_test = df_test.iloc[:, 1]

In [96]:
print(X_test.shape)
print(y_test.shape)

(7493, 49)
(7493,)


### 正規化

* 輸入 X 轉成 Numpy Array 

In [98]:
X_train = np.array(X_train).astype(float)
X_test = np.array(X_test).astype(float)
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")

X_train shape: (17793, 49)
X_test shape: (7493, 49)


In [100]:
# normalize
from sklearn import preprocessing
quantile_transformer = preprocessing.QuantileTransformer(random_state=17)
X_train = quantile_transformer.fit_transform(X_train)
X_test = quantile_transformer.fit_transform(X_test)
X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")

ValueError: ignored

* 目標 y 轉成 Numpy Array 

In [101]:
le = LabelEncoder()
le.fit(y_train)
print('label class: ',le.classes_)

y_train = le.transform(y_train)
y_test = le.transform(y_test)

label class:  ['unworn' 'worn']


In [102]:
y_train = np.array(y_train).astype(float).reshape(-1,1)
print(f"y_train shape: {y_train.shape}")
y_test = np.array(y_test).astype(float).reshape(-1,1)
print(f"y_test shape: {y_test.shape}")

y_train shape: (17793, 1)
y_test shape: (7493, 1)


In [103]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train, random_state=17)

### 模型超參數

In [105]:
max_features = 500
maxlen = X_train.shape[1]
# maxlen = 20
batch_size = 32

## 設計 TCN (Temporal Convolutional Network) 模型

In [106]:
from tcn import compiled_tcn
model = compiled_tcn(return_sequences=False,
                         num_feat=1,
                         num_classes=1,
                         nb_filters=10,
                         kernel_size=10,
                         dilations=[1, 2, 4, 8, 16, 32],
                         nb_stacks=5,
                         max_len=X_train[0:1].shape[1],
                         use_skip_connections=True)
model.summary()

x.shape= (None, 10)
model.x = (None, 49, 1)
model.y = (None, 1)
Model: "functional_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        [(None, 49, 1)]           0         
_________________________________________________________________
tcn (TCN)                    (None, 10)                59720     
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 11        
_________________________________________________________________
activation_11 (Activation)   (None, 1)                 0         
Total params: 59,731
Trainable params: 59,731
Non-trainable params: 0
_________________________________________________________________


## 模型訓練

In [110]:
history = model.fit(X_train, y_train, epochs=15)

Epoch 1/15
557/557 [==============================] - 18s 32ms/step - loss: nan - accuracy: 0.5485
Epoch 2/15
135/557 [======>.......................] - ETA: 12s - loss: nan - accuracy: 0.5431

KeyboardInterrupt: ignored

In [111]:
result = model.evaluate(X_train, y_train)
print(f"Accuracy : {result[1] * 100:.2f} %")
print(f"Loss : {result[0] * 100:.2f} %")

557/557 [==============================] - 4s 7ms/step - loss: nan - accuracy: 0.4515
Accuracy : 45.15 %
Loss : nan %


In [54]:
# from tcn import compiled_tcn
# model = compiled_tcn(return_sequences=False,
#                          num_feat=1,
#                          num_classes=1,
#                          nb_filters=128,
#                          kernel_size=2,
#                          dilations=[1,2,4,8,16],
#                          nb_stacks=1,
#                          max_len=X_train[0:1].shape[1],
#                          use_skip_connections=True)

In [64]:
# i = Input(shape = (maxlen,))
# x = Embedding(max_features, 128)(i)
# x = TCN(nb_filters = 64, kernel_size=2, dilations=[1,2,4,8,16])(x)
# # x = Dropout(0.01)(x)
# x = Dense(1, activation = 'sigmoid')(x)
# model = Model(inputs=[i], outputs=[x])
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])
# # model.compile(optimizer='adam', metrics = ['accuracy'])
# model.summary()

### 模型訓練

In [65]:
# history = model.fit(X_train, y_train, batch_size = batch_size, epochs = 10)
# history = model.fit(X_train, y_train, batch_size = batch_size, epochs = 10, validation_data = [X_test, y_test])
# model.fit(X_train, y_train, epochs=100,validation_data=(X_test, y_test))